1. Load the Required Libraries

In [1]:
import pandas as pd
import numpy as np
import dtale
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier as eXTremeGradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_auc_score,recall_score

2. Read the data from the dataset

In [2]:
df = pd.read_csv("D:\\ResearchPapers\\Datasets\\diabetes_dataset.csv", encoding='latin1')
d=dtale.show(df)
d.open_browser()

In [3]:
df.head()
df.drop(columns=['year','location','race:AfricanAmerican','race:Asian','race:Caucasian','race:Hispanic','race:Other'], inplace=True)

3. Handling missing values and replacing missing values with nan from numpy and replace with mean of all the other values

In [4]:
print(df.isnull().sum())
print(df.isna().sum())
dtale.show(df)

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
hbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64
gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
hbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64


In [5]:
# Check unique values in categorical columns
print(df['smoking_history'].unique())
print(df['gender'].unique())  

['never' 'not current' 'current' 'No Info' 'ever' 'former']
['Female' 'Male' 'Other']


In [6]:
print(df['age'].describe())       # Check min/max age
print(df['bmi'].min())            # Check if BMI is 0 or negative

count    100000.000000
mean         41.885856
std          22.516840
min           0.080000
25%          24.000000
50%          43.000000
75%          60.000000
max          80.000000
Name: age, dtype: float64
10.01


4. Encoding the categorical data

In [7]:
label_encoder = LabelEncoder()
df['gender'] = label_encoder.fit_transform(df['gender'])
dtale.show(df)

In [8]:
le = LabelEncoder()
df['smoking_history'] = le.fit_transform(df['smoking_history'].astype(str))
dtale.show(df)

4. Split the attribites into dependent and independent attributes

In [18]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values
dtale.show(Y, ignore_duplicate=True)

5. Splitting the dataset intro training set and test set

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

6. Train the Random Forest Model

In [20]:
lf = eXTremeGradientBoostingClassifier(max_iter=1000, random_state=42)
lf.fit(X_train, Y_train)
Kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(lf, X_train, Y_train, cv=Kfold, scoring='accuracy')
Y_pred = lf.predict(X_test)

C:\Users\Portia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:183: UserWarning:

[01:33:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "max_iter" } are not used.


C:\Users\Portia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:183: UserWarning:

[01:33:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "max_iter" } are not used.


C:\Users\Portia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:183: UserWarning:

[01:33:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "max_iter" } are not used.


C:\Users\Portia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\L

In [21]:
lf.score(X_test, Y_test)
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98     18294
           1       0.95      0.70      0.80      1706

    accuracy                           0.97     20000
   macro avg       0.96      0.85      0.89     20000
weighted avg       0.97      0.97      0.97     20000



In [22]:
lf2 = eXTremeGradientBoostingClassifier(
    
   max_iter=1000,    random_state=42)

8. Retraining the model using Resampled data

In [23]:
smote_enn = SMOTE(sampling_strategy='minority', random_state=42)
X_train_res, Y_train_res = smote_enn.fit_resample(X_train, Y_train)
print(pd.Series(Y_train_res).value_counts())

0    73206
1    73206
Name: count, dtype: int64


In [24]:
model = eXTremeGradientBoostingClassifier( random_state= 42)
model.fit(X_train_res, Y_train_res) 
Kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(model, X_train_res, Y_train_res, cv=Kfold, scoring='accuracy')

In [25]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]
sensitivity = recall_score(Y_test, y_pred, pos_label=1)
roc_auc = roc_auc_score(Y_test, y_proba)

tn, fp, fn, tp = confusion_matrix(Y_test, y_pred).ravel()
specificity = tn / (tn + fp)


report = classification_report(Y_test, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()

pd.set_option("display.precision", 4)
print(df_report)
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Confusion Matrix:\n{confusion_matrix(Y_test, y_pred)}")
print(f"Mean Accuracy:{scores.mean():.4f} (+/- {scores.std():.4f})")

              precision  recall  f1-score     support
0                0.9730  0.9964    0.9846  18294.0000
1                0.9487  0.7040    0.8082   1706.0000
accuracy         0.9715  0.9715    0.9715      0.9715
macro avg        0.9609  0.8502    0.8964  20000.0000
weighted avg     0.9710  0.9715    0.9696  20000.0000
ROC-AUC: 0.9781
Sensitivity: 0.7040
Specificity: 0.9964
Confusion Matrix:
[[18229    65]
 [  505  1201]]
Mean Accuracy:0.9815 (+/- 0.0011)
